In [1]:
import numpy as np
from time import time
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import mahalanobis

class FaceRecognitionModel:
    def __init__(self, data_path='D:\\Images', n_components=50):
        self.data_path = data_path
        self.n_components = n_components
        self.images = []
        self.labels = []
        self.x_train, self.x_test, self.y_train, self.y_test = None, None, None, None
        self.pca = None
        self.clf = None
        self.cov_matrix = None
        self.inv_cov_matrix = None

    def load_data(self):
        for i in range(1, 41):
            for j in range(1, 11):
                file_path = os.path.join(self.data_path,'s'+str(i),str(j)+'.pgm')
                img = cv2.imread(file_path, 0)
                img = img.flatten()
                self.images.append(img)
                self.labels.append(i)

        self.images = np.array(self.images)
        self.labels = np.array(self.labels)

    def split_data(self):
        x_train, x_test, y_train, y_test = [], [], [], []
        for i in range(1, 41):
            indices = np.where(self.labels == i)[0]
            train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

            x_train.extend(self.images[train_indices])
            y_train.extend(self.labels[train_indices])

            x_test.extend(self.images[test_indices])
            y_test.extend(self.labels[test_indices])

        self.x_train, self.y_train = np.array(x_train), np.array(y_train)
        self.x_test, self.y_test = np.array(x_test), np.array(y_test)
    
    def plot_elbow(self):
        X_std = (self.x_train - np.mean(self.x_train, axis=0)) / np.std(self.x_train, axis=0)
        pca = PCA()
        pca.fit(X_std)
        explained_variance_ratio = pca.explained_variance_ratio_
        cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

        plt.plot(range(1, len(explained_variance_ratio) + 1), cumulative_variance_ratio, marker='o')
        plt.title('Elbow Chart for PCA')
        plt.xlabel('Number of Principal Components')
        plt.ylabel('Cumulative Explained Variance Ratio')
        plt.show()

    def perform_pca(self):
        print(f"Extracting the top {self.n_components} eigenfaces from {self.x_train.shape[0]} faces")
        t0 = time()
        self.pca = PCA(n_components=self.n_components, svd_solver='randomized', whiten=True).fit(self.x_train)
        print("Done in %0.3fs" % (time() - t0))

    def project_on_eigenfaces(self):
        print("Projecting the input data on the eigenfaces orthonormal basis")
        t0 = time()
        self.x_train = self.pca.transform(self.x_train)
        self.x_test = self.pca.transform(self.x_test)
        print("Done in %0.3fs" % (time() - t0))
        
    def mahalabonis(self,weight = 'uniform'):
        print("Predicting the people names on the testing set")
        t0 = time()
        self.cov_matrix = np.cov(self.x_train, rowvar=False)
        self.inv_cov_matrix = np.linalg.inv(self.cov_matrix)
        distances = np.array([[mahalanobis(test_img, train_img, self.inv_cov_matrix)\
                                      for train_img in self.x_train] for test_img in self.x_test])
        if weight.lower() == 'uniform':
            weight = 1.0
        elif weight.lower() == 'distance':
            weight = 1.0 / (distances + 1e-10)
        distances = weight * distances
        five_neighbors_dis = np.argsort(distances, axis=1)[:, :5]
        pred = np.array([np.argmax(np.bincount(self.y_train[idx])) for idx in five_neighbors_dis])
        print("Done in %0.3fs" % (time() - t0))
        # Evaluate accuracy
        print("\nClassification Report:")
        print(classification_report(self.y_test, pred))
        print('Accuracy:', accuracy_score(self.y_test,pred))

    def uniform_weight(distance):
        return 1.0

    def distance_weight(distance):
        return 1.0 / (distance + 1e-10)  # Adding a small epsilon to avoid division by zero
  
    def train_classifier(self,classifier):
        print("Fitting the classifier to the training set")
        t0 = time()
        self.clf = classifier
        self.clf.fit(self.x_train, self.y_train)
        print("Done in %0.3fs" % (time() - t0))
        
    def print_classifier_info(self):
        print("Classifier:", self.clf)
        print("Hyperparameters:", self.clf.get_params())
        
    def evaluate_model(self):
        print("Predicting the people names on the testing set")
        t0 = time()
        y_pred = self.clf.predict(self.x_test)
        print("Done in %0.3fs" % (time() - t0))
        
        print("\nClassification Report:")
        print(classification_report(self.y_test, y_pred))
        print('Accuracy:', accuracy_score(self.y_test, y_pred))
        # Print only overall stats recall and f1 score
        # print('Recall: ', recall_score(self.y_test, y_pred, average='macro'))
        # print('F1 score: ', f1_score(self.y_test, y_pred, average='macro'))

    def plot_eigenfaces(self):
        eigenfaces = self.pca.components_.reshape((self.n_components, 112, 92))

        plt.figure(figsize=(16, 9))
        for i in range(self.n_components):
            plt.subplot(10, 10, i + 1)
            plt.imshow(eigenfaces[i], cmap='gray')
            plt.xticks(())
            plt.yticks(())
        plt.show()
if __name__ == '__main__':
    clf = SVC(kernel='rbf', class_weight='balanced')
    knn = KNeighborsClassifier(n_neighbors=5, weights = 'uniform', metric = 'euclidean')
    face_model = FaceRecognitionModel()
    face_model.load_data()
    face_model.split_data()
    face_model.perform_pca()
    face_model.project_on_eigenfaces()
    #     face_model.plot_eigenfaces()
    face_model.train_classifier(classifier=knn)
    face_model.print_classifier_info()
    face_model.evaluate_model()
    

C:\Users\PC\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting the top 50 eigenfaces from 320 faces
Done in 0.384s
Projecting the input data on the eigenfaces orthonormal basis
Done in 0.061s
Fitting the classifier to the training set
Done in 0.002s
Classifier: KNeighborsClassifier(metric='euclidean')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Predicting the people names on the testing set
Done in 0.127s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         2
          

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'uniform',metric='manhattan')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.001s
Classifier: KNeighborsClassifier(metric='manhattan')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Predicting the people names on the testing set
Done in 0.008s

Classification Report:
              precision    recall  f1-score   support

           1       0.67      1.00      0.80         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.50      1.00      0.67         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.0

In [3]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'uniform',metric='cosine')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.001s
Classifier: KNeighborsClassifier(metric='cosine')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'cosine', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Predicting the people names on the testing set
Done in 0.005s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       0.67      1.00      0.80         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00     

In [4]:
face_model.mahalabonis(weight = 'uniform')

Predicting the people names on the testing set
Done in 0.264s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       0.67      1.00      0.80         2
          16       1.00      0.50      0.67      

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Weight Distance

In [5]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'distance',metric='euclidean')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.001s
Classifier: KNeighborsClassifier(metric='euclidean', weights='distance')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Predicting the people names on the testing set
Done in 0.010s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.

In [6]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'distance',metric='manhattan')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.002s
Classifier: KNeighborsClassifier(metric='manhattan', weights='distance')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Predicting the people names on the testing set
Done in 0.010s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       0.67      1.00      0.80         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.

In [7]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'distance',metric='cosine')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.001s
Classifier: KNeighborsClassifier(metric='cosine', weights='distance')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'cosine', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Predicting the people names on the testing set
Done in 0.009s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       0.67      1.00      0.80         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         2
          11       1.00    

In [8]:
face_model.mahalabonis(weight = 'distance')

Predicting the people names on the testing set
Done in 0.282s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       0.67      1.00      0.80         2
          16       1.00      0.50      0.67      

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SVM

In [9]:
clf = SVC(kernel='rbf', class_weight='balanced')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.045s
Classifier: SVC(class_weight='balanced')
Hyperparameters: {'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': 'balanced', 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Predicting the people names on the testing set
Done in 0.006s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00  

# K = 70, weight = uniform

In [10]:
if __name__ == '__main__':
    clf = SVC(kernel='rbf', class_weight='balanced')
    knn = KNeighborsClassifier(n_neighbors=5, weights = 'uniform', metric = 'euclidean')
    face_model = FaceRecognitionModel(n_components=70)
    face_model.load_data()
    face_model.split_data()
    face_model.perform_pca()
    face_model.project_on_eigenfaces()
    #     face_model.plot_eigenfaces()
    face_model.train_classifier(classifier=knn)
    face_model.print_classifier_info()
    face_model.evaluate_model()

Extracting the top 70 eigenfaces from 320 faces
Done in 0.392s
Projecting the input data on the eigenfaces orthonormal basis
Done in 0.071s
Fitting the classifier to the training set
Done in 0.001s
Classifier: KNeighborsClassifier(metric='euclidean')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Predicting the people names on the testing set
Done in 0.007s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.29      1.00      0.44         2
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         2
          

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'uniform',metric='manhattan')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.002s
Classifier: KNeighborsClassifier(metric='manhattan')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Predicting the people names on the testing set
Done in 0.011s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.33      1.00      0.50         2
           6       0.67      1.00      0.80         2
           7       0.67      1.00      0.80         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.0

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'uniform',metric='cosine')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.002s
Classifier: KNeighborsClassifier(metric='cosine')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'cosine', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Predicting the people names on the testing set
Done in 0.007s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       0.67      1.00      0.80         2
           3       1.00      1.00      1.00         2
           4       0.67      1.00      0.80         2
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00     

In [13]:
face_model.mahalabonis(weight = 'uniform')

Predicting the people names on the testing set
Done in 0.244s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.29      1.00      0.44         2
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         2
          14       1.00      0.50      0.67         2
          15       0.67      1.00      0.80         2
          16       0.00      0.00      0.00      

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# k = 70 ,weight = distance

In [14]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'distance',metric='euclidean')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.001s
Classifier: KNeighborsClassifier(metric='euclidean', weights='distance')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Predicting the people names on the testing set
Done in 0.009s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.50      1.00      0.67         2
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.

In [15]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'distance',metric='manhattan')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.001s
Classifier: KNeighborsClassifier(metric='manhattan', weights='distance')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Predicting the people names on the testing set
Done in 0.008s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.40      1.00      0.57         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
clf = KNeighborsClassifier(n_neighbors=5, weights= 'distance',metric='cosine')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.001s
Classifier: KNeighborsClassifier(metric='cosine', weights='distance')
Hyperparameters: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'cosine', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Predicting the people names on the testing set
Done in 0.006s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00    

In [17]:
face_model.mahalabonis(weight = 'distance')

Predicting the people names on the testing set
Done in 0.258s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.29      1.00      0.44         2
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00         2
          14       1.00      0.50      0.67         2
          15       0.67      1.00      0.80         2
          16       0.00      0.00      0.00      

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
clf = SVC(kernel='rbf', class_weight='balanced')
face_model.train_classifier(classifier=clf)
face_model.print_classifier_info()
face_model.evaluate_model()

Fitting the classifier to the training set
Done in 0.066s
Classifier: SVC(class_weight='balanced')
Hyperparameters: {'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': 'balanced', 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Predicting the people names on the testing set
Done in 0.011s

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00  